In [7]:
#!pip install jax
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import scipy.integrate as spi

from scipy.optimize import brentq, fsolve
from scipy.integrate import quad
import time
import os

import tensorflow as tf
import tensornetwork as tn

# Set TensorNetwork to use the TensorFlow backend (needed for autodiff)
# tn.set_default_backend("tensorflow")
tn.set_default_backend("jax")


In [13]:
import tensorflow as tf
import tensornetwork as tn
import numpy as np

# Number of qubits
N = 5  # number of qubits

# Each site tensor has shape (chi_left=1, d=2, chi_right=1)
# Initialize all qubits in |0>, i.e., A[0]=1, A[1]=0
prod_tensors = []
for _ in range(N):
    # shape (2,) → reshape into (1,2,1) to represent a single qubit in the |0> state
    v = tf.constant([1.0, 0.0], dtype=tf.float32)  # Use real type, not complex
    # Convert to numpy and append (ensure tensor shape is integer-based)
    prod_tensors.append(v.numpy())  # Convert to numpy array

# Initialize FiniteMPS (in canonical form)
psi_initial = tn.FiniteMPS(prod_tensors, canonicalize=True)

# Step 1: Define the Hadamard gate (in real numbers)
Hgate = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.float32) / tf.cast(tf.sqrt(2.0), tf.float32)

# Step 2: Apply the Hadamard gate to the first qubit (on the first site)
A0 = psi_initial.tensors[0]  # shape (1,2,1) for the first qubit

# Contract the Hadamard gate with the physical index of A0
# Tensor contraction over the physical index (index 1 of A0)
A0 = tf.tensordot(Hgate, A0, axes=([1], [1]))  # shape (2,1,1)

# Step 3: Update the MPS tensor for the first site
# We need to transpose it back to the canonical form (1,2,1)
psi_initial.tensors[0] = tf.transpose(A0, (1, 0, 2))


TypeError: Shapes must be 1D sequences of concrete values of integer type, got [2, 1.0].

In [ ]:
#Two-qubit unitary
I = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j,  1 + 0j]
    ], dtype=tf.complex64)
X = tf.constant([
        [0 + 0j, 1 + 0j],
        [1 + 0j,  0 + 0j]
    ], dtype=tf.complex64)
Y = tf.constant([
        [0 + 0j, 0 - 1j],
        [0 + 1j,  0 + 0j]
    ], dtype=tf.complex64)
Z = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j,  -1 + 0j]
    ], dtype=tf.complex64)

# 15 Hermitian generator basis: 
# 9 two-site interactions + 3 local on qubit 1 + 3 local on qubit 2
paulis = [X, Y, Z]
basis_list = []

# Interaction terms sigma^a ⊗ sigma^b (a,b in {X,Y,Z})
for A in paulis:
    for B in paulis:
        basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(A),
                                                              tf.linalg.LinearOperatorFullMatrix(B)])
                          .to_dense())


# Single-qubit rotations on 1: sigma^a ⊗ I
for A in paulis:
    basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(A),
                                                          tf.linalg.LinearOperatorFullMatrix(I)])
                      .to_dense())

# Single-qubit rotations on 2: I ⊗ sigma^a
for B in paulis:
    basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(I),
                                                          tf.linalg.LinearOperatorFullMatrix(B)])
                      .to_dense())

# Stack into a (15,4,4) tensor
basis = tf.stack(basis_list, axis=0)  # shape (15,4,4)


@tf.function
def make_two_qubit_unitary(theta):
    # theta: real tf.Tensor shape (15,)
    theta_c = tf.cast(theta, tf.complex64)               # → complex64
    H = tf.tensordot(theta_c, basis, axes=([0],[0]))    # shape (4,4)
    U_flat = tf.linalg.expm(-1j * H)                     # shape (4,4)
    return tf.reshape(U_flat, [2,2,2,2])                 # shape (2,2,2,2)



theta = tf.Variable(tf.random.normal([15], dtype=tf.float32))
U_gate = make_two_qubit_unitary(theta)

# U_gate is ready to apply in your two-site update
print("Two-qubit gate shape:", U_gate.shape)

Two-qubit gate shape: (2, 2, 2, 2)


In [ ]:
#Cost function
mpo_tensors = []
for _ in range(N):
    # now (phys_in=2, phys_out=2, χ_left=1, χ_right=1)
    t = tf.reshape(X, (1, 1, 2, 2))
    mpo_tensors.append(t)

mpo_X = tn.FiniteMPO(mpo_tensors)


NameError: name 'N' is not defined

In [ ]:

mps_mpo_expectation_value(psi_initial, mpo_X)
print("⟨X⟩ =", expval.numpy())

NameError: name 'mps_mpo_expectation_value' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensornetwork as tn

# Set TensorNetwork to use the TensorFlow backend (needed for autodiff)
tn.set_default_backend("tensorflow")

def copy_mps(mps):
    """
    Create a deep copy of a FiniteMPS object.
    
    Args:
        mps: FiniteMPS object
        
    Returns:
        New FiniteMPS object with the same tensors
    """
    # Copy all tensors
    tensor_copies = [tf.identity(tensor) for tensor in mps.tensors]
    
    # Create a new MPS with the copied tensors
    new_mps = tn.FiniteMPS(tensor_copies, canonicalize=False)
    
    # Copy the center position
    new_mps.center_position = mps.center_position
    
    return new_mps

def initialize_mps(N, initial_state="zero"):
    """
    Initialize an MPS with N qubits in the specified initial state.
    
    Args:
        N: Number of qubits
        initial_state: 'zero' for |0⟩^⊗N, 'plus' for |+⟩^⊗N
    
    Returns:
        FiniteMPS object
    """
    prod_tensors = []
    
    if initial_state == "zero":
        # Initialize all qubits in |0⟩
        for _ in range(N):
            v = tf.constant([1.0, 0.0], dtype=tf.complex64)
            prod_tensors.append(tf.reshape(v, (1, 2, 1)))
    elif initial_state == "plus":
        # Initialize all qubits in |+⟩
        v = tf.constant([1.0, 1.0], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
        for _ in range(N):
            prod_tensors.append(tf.reshape(v, (1, 2, 1)))
    else:
        raise ValueError("Unsupported initial state")
    
    # Initialize FiniteMPS (in canonical form)
    return tn.FiniteMPS(prod_tensors, canonicalize=True)

def apply_single_qubit_gate(mps, gate, site):
    """
    Apply a single-qubit gate to a specific site of the MPS.
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2] tensor representing the single-qubit gate
        site: Site index to apply the gate to
    
    Returns:
        Updated FiniteMPS object
    """
    # Make a copy to avoid modifying the original
    mps_copy = copy_mps(mps)
    
    # Apply gate to the specified site
    tensor = mps_copy.tensors[site]  # shape (chi_left, 2, chi_right)
    gate_applied = tf.tensordot(gate, tensor, axes=([1], [1]))  # (2, chi_left, chi_right)
    mps_copy.tensors[site] = tf.transpose(gate_applied, (1, 0, 2))  # (chi_left, 2, chi_right)
    
    return mps_copy

def apply_two_qubit_gate(mps, gate, site1, site2, max_bond_dim=None):
    """
    Apply a two-qubit gate to adjacent sites in the MPS.
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2, 2, 2] tensor representing the two-qubit gate
        site1, site2: Adjacent site indices
        max_bond_dim: Maximum bond dimension after SVD truncation
    
    Returns:
        Updated FiniteMPS object
    """
    # Ensure sites are adjacent
    if abs(site1 - site2) != 1:
        raise ValueError("Sites must be adjacent")
    
    # Make sure site1 < site2
    if site1 > site2:
        site1, site2 = site2, site1
        # Permute gate indices to match the new site order
        gate = tf.transpose(gate, [2, 3, 0, 1])
    
    # Make a copy of the MPS
    mps_copy = copy_mps(mps)
    
    # Ensure the MPS is in the right canonical form
    mps_copy.position(site1)
    
    # Get tensors at the two sites
    tensor1 = mps_copy.tensors[site1]  # (chi_left1, 2, chi_right1)
    tensor2 = mps_copy.tensors[site2]  # (chi_left2, 2, chi_right2)
    
    # Extract dimensions
    chi_left1, d1, chi_right1 = tensor1.shape
    chi_left2, d2, chi_right2 = tensor2.shape
    
    # Contract tensors to form two-site tensor
    theta = tf.reshape(tensor1, [chi_left1, d1 * chi_right1])
    theta = tf.matmul(theta, tf.reshape(tensor2, [chi_right1, d2 * chi_right2]))
    theta = tf.reshape(theta, [chi_left1, d1, d2, chi_right2])
    
    # Apply the gate
    theta_prime = tf.einsum('ijkl,jknm->inml', theta, gate)
    theta_prime = tf.reshape(theta_prime, [chi_left1 * d1, d2 * chi_right2])
    
    # SVD to decompose back to MPS form
    s, u, v = tf.linalg.svd(theta_prime, full_matrices=False)
    
    # Truncate if needed
    if max_bond_dim is not None and s.shape[0] > max_bond_dim:
        s = s[:max_bond_dim]
        u = u[:, :max_bond_dim]
        v = v[:max_bond_dim, :]
    
    # Create new tensors
    chi_new = s.shape[0]
    
    # Include singular values in the u tensor
    u = u * tf.reshape(tf.sqrt(s), [1, -1])
    v = v * tf.reshape(tf.sqrt(s), [-1, 1])
    
    # Reshape back to rank-3 tensors
    new_tensor1 = tf.reshape(u, [chi_left1, d1, chi_new])
    new_tensor2 = tf.reshape(v, [chi_new, d2, chi_right2])
    
    # Update the tensors in the MPS
    mps_copy.tensors[site1] = new_tensor1
    mps_copy.tensors[site2] = new_tensor2
    
    # Update center position
    mps_copy.center_position = site2
    
    return mps_copy

def apply_sequential_two_qubit_gates(mps, gate, max_bond_dim=None):
    """
    Apply a two-qubit gate sequentially to all adjacent pairs of qubits.
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2, 2, 2] tensor representing the two-qubit gate
        max_bond_dim: Maximum bond dimension after SVD truncation
    
    Returns:
        Updated FiniteMPS object
    """
    mps_result = copy_mps(mps)
    N = len(mps_result.tensors)
    
    # Apply gate to each adjacent pair: (0,1), (1,2), ... (N-2,N-1)
    for i in range(N - 1):
        mps_result = apply_two_qubit_gate(mps_result, gate, i, i + 1, max_bond_dim)
    
    return mps_result

def create_pauli_mpo(N, operator_type):
    """
    Create an MPO for a Pauli operator on all sites or nearest-neighbor interactions.
    
    Args:
        N: Number of qubits
        operator_type: String identifier for the observable ('X', 'Z', 'ZZ_nn')
        
    Returns:
        FiniteMPO object
    """
    # Define Pauli operators
    I = tf.constant([[1.0, 0.0], [0.0, 1.0]], dtype=tf.complex64)
    X = tf.constant([[0.0, 1.0], [1.0, 0.0]], dtype=tf.complex64)
    Y = tf.constant([[0.0, -1j], [1j, 0.0]], dtype=tf.complex64)
    Z = tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64)
    
    mpo_tensors = []
    
    if operator_type in ['X', 'Y', 'Z']:
        # Select the appropriate operator
        op = {'X': X, 'Y': Y, 'Z': Z}[operator_type]
        
        # Create MPO for single-site operator on every site
        for _ in range(N):
            # Shape: (bond_left=1, bond_right=1, phys_in=2, phys_out=2)
            t = tf.reshape(op, (1, 1, 2, 2))
            mpo_tensors.append(t)
    
    elif operator_type == 'ZZ_nn':
        # MPO for sum of nearest-neighbor ZZ interactions
        
        # First site: [I, Z]
        first_site = tf.zeros((1, 2, 2, 2), dtype=tf.complex64)
        first_site = tf.tensor_scatter_nd_update(
            first_site, 
            [[0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]],
            [tf.complex(1.0, 0.0), tf.complex(1.0, 0.0), tf.complex(1.0, 0.0), tf.complex(-1.0, 0.0)]
        )
        mpo_tensors.append(first_site)
        
        # Middle sites: [I, 0; Z, I]
        for _ in range(N - 2):
            mid_site = tf.zeros((2, 2, 2, 2), dtype=tf.complex64)
            mid_site = tf.tensor_scatter_nd_update(
                mid_site,
                [[0, 0, 0, 0], [0, 0, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 1, 1]],
                [tf.complex(1.0, 0.0), tf.complex(1.0, 0.0), tf.complex(1.0, 0.0), 
                 tf.complex(-1.0, 0.0), tf.complex(1.0, 0.0), tf.complex(1.0, 0.0)]
            )
            mpo_tensors.append(mid_site)
        
        # Last site: [Z, I]
        last_site = tf.zeros((2, 1, 2, 2), dtype=tf.complex64)
        last_site = tf.tensor_scatter_nd_update(
            last_site,
            [[0, 0, 0, 0], [0, 0, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1]],
            [tf.complex(1.0, 0.0), tf.complex(-1.0, 0.0), tf.complex(1.0, 0.0), tf.complex(1.0, 0.0)]
        )
        mpo_tensors.append(last_site)
    
    else:
        raise ValueError(f"Unsupported operator type: {operator_type}")
    
    return tn.FiniteMPO(mpo_tensors)

def mps_mpo_expectation_value(mps, mpo):
    """
    Calculate the expectation value <ψ|O|ψ> for an MPS and MPO.
    
    Args:
        mps: FiniteMPS object
        mpo: FiniteMPO object
        
    Returns:
        Real-valued expectation value
    """
    # Contract the MPS with the MPO and the conjugate of the MPS
    mps_prime = mpo.apply(mps)
    result = mps.inner_product(mps_prime)
    
    # Ensure the result is real (should be, for Hermitian observables)
    return tf.math.real(result)

def make_two_qubit_unitary(theta, basis):
    """
    Create a two-qubit unitary gate from parameters and a basis.
    
    Args:
        theta: Parameters, shape (15,)
        basis: Basis of generators, shape (15, 4, 4)
    
    Returns:
        Unitary gate as a tensor of shape (2, 2, 2, 2)
    """
    # Convert parameters to complex
    theta_c = tf.cast(theta, tf.complex64)
    
    # Generate Hermitian generator
    H = tf.tensordot(theta_c, basis, axes=([0], [0]))  # shape (4, 4)
    
    # Create unitary via matrix exponential
    U_flat = tf.linalg.expm(-1j * H)  # shape (4, 4)
    
    # Reshape to two-qubit gate format
    return tf.reshape(U_flat, [2, 2, 2, 2])  # shape (2, 2, 2, 2)

def create_basis():
    """
    Create the basis of 15 generators for the two-qubit unitary.
    
    Returns:
        Tensor of shape (15, 4, 4)
    """
    # Define Pauli matrices
    I = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j, 1 + 0j]
    ], dtype=tf.complex64)
    X = tf.constant([
        [0 + 0j, 1 + 0j],
        [1 + 0j, 0 + 0j]
    ], dtype=tf.complex64)
    Y = tf.constant([
        [0 + 0j, 0 - 1j],
        [0 + 1j, 0 + 0j]
    ], dtype=tf.complex64)
    Z = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j, -1 + 0j]
    ], dtype=tf.complex64)
    
    paulis = [X, Y, Z]
    basis_list = []
    
    # Interaction terms sigma^a ⊗ sigma^b (a,b in {X,Y,Z})
    for A in paulis:
        for B in paulis:
            basis_list.append(tf.linalg.LinearOperatorKronecker([
                tf.linalg.LinearOperatorFullMatrix(A),
                tf.linalg.LinearOperatorFullMatrix(B)
            ]).to_dense())
    
    # Single-qubit rotations on 1: sigma^a ⊗ I
    for A in paulis:
        basis_list.append(tf.linalg.LinearOperatorKronecker([
            tf.linalg.LinearOperatorFullMatrix(A),
            tf.linalg.LinearOperatorFullMatrix(I)
        ]).to_dense())
    
    # Single-qubit rotations on 2: I ⊗ sigma^a
    for B in paulis:
        basis_list.append(tf.linalg.LinearOperatorKronecker([
            tf.linalg.LinearOperatorFullMatrix(I),
            tf.linalg.LinearOperatorFullMatrix(B)
        ]).to_dense())
    
    # Stack into a (15, 4, 4) tensor
    return tf.stack(basis_list, axis=0)

def cost_function(theta, basis, N, observable_type, max_bond_dim=None):
    """
    Calculate the expectation value of an observable for a quantum state
    evolved by the parameterized circuit.
    
    Args:
        theta: Parameters of the two-qubit gate, shape (15,)
        basis: Basis of generators, shape (15, 4, 4)
        N: Number of qubits
        observable_type: Type of observable to measure
        max_bond_dim: Maximum bond dimension for MPS
        
    Returns:
        Expectation value (to be minimized)
    """
    # Create the two-qubit unitary gate
    U_gate = make_two_qubit_unitary(theta, basis)
    
    # Initialize MPS in |0⟩^⊗N state
    mps = initialize_mps(N, initial_state="zero")
    
    # Apply Hadamard to the first qubit (optional)
    H_gate = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
    mps = apply_single_qubit_gate(mps, H_gate, 0)
    
    # Apply the two-qubit gate sequentially
    mps = apply_sequential_two_qubit_gates(mps, U_gate, max_bond_dim)
    
    # Create the observable MPO
    mpo = create_pauli_mpo(N, observable_type)
    
    # Calculate the expectation value
    expectation = mps_mpo_expectation_value(mps, mpo)
    
    return expectation

def optimize_circuit(N, observable_type='ZZ_nn', max_bond_dim=None, 
                     learning_rate=0.01, iterations=100):
    """
    Optimize the parameters of the two-qubit gate to minimize the expectation
    value of the given observable.
    
    Args:
        N: Number of qubits
        observable_type: Type of observable to measure
        max_bond_dim: Maximum bond dimension for MPS
        learning_rate: Learning rate for the optimizer
        iterations: Number of optimization steps
        
    Returns:
        Optimized parameters and history of expectation values
    """
    # Create basis for the two-qubit unitary
    basis = create_basis()
    
    # Initialize parameters randomly
    theta = tf.Variable(tf.random.normal([15], stddev=0.1, dtype=tf.float32))
    
    # Create optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    # Store the history of expectation values
    history = []
    
    # Run optimization
    for step in range(iterations):
        with tf.GradientTape() as tape:
            # Calculate expectation value (cost function)
            expectation = cost_function(theta, basis, N, observable_type, max_bond_dim)
        
        # Calculate gradients
        gradients = tape.gradient(expectation, [theta])
        
        # Apply gradients
        optimizer.apply_gradients(zip(gradients, [theta]))
        
        # Store expectation value
        history.append(expectation.numpy())
        
        # Print progress
        if step % 10 == 0 or step == iterations - 1:
            print(f"Step {step}: Expectation value = {expectation.numpy():.6f}")
    
    return theta.numpy(), history

def visualize_results(history):
    """
    Visualize the optimization history.
    
    Args:
        history: List of expectation values
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history)
    plt.grid(True)
    plt.xlabel('Optimization Step')
    plt.ylabel('Expectation Value')
    plt.title('Optimization Progress')
    plt.show()

def main():
    """
    Main function to run the optimization.
    """
    # Set parameters
    N = 5  # number of qubits
    observable_type = 'ZZ_nn'  # nearest-neighbor ZZ interactions
    max_bond_dim = 16  # maximum bond dimension
    learning_rate = 0.01
    iterations = 200
    
    print(f"Starting optimization for {N}-qubit system")
    print(f"Observable: {observable_type}")
    print(f"Maximum bond dimension: {max_bond_dim}")
    
    # Run optimization
    optimal_params, history = optimize_circuit(
        N, observable_type, max_bond_dim, learning_rate, iterations
    )
    
    print(f"Final expectation value: {history[-1]:.6f}")
    print(f"Optimal parameters: {optimal_params}")
    
    # Visualize results
    visualize_results(history)
    
    # Create the optimal unitary
    basis = create_basis()
    optimal_unitary = make_two_qubit_unitary(optimal_params, basis)
    
    print("Optimal two-qubit unitary (real part):")
    print(tf.reshape(tf.math.real(optimal_unitary), (4, 4)).numpy())



main()

Starting optimization for 5-qubit system
Observable: ZZ_nn
Maximum bond dimension: 16


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,2], In[1]: [1,2] [Op:MatMul] name: 

In [ ]:
import numpy as np
import tensorflow as tf
import tensornetwork as tn

# Set TensorNetwork to use TensorFlow backend
tn.set_default_backend("tensorflow")

def initialize_simple_mps(n_qubits):
    """
    Initialize a simple MPS with all qubits in |0⟩
    """
    # Create tensors for MPS in |0⟩^⊗n state
    tensors = []
    for _ in range(n_qubits):
        # Create tensor for |0⟩ state: [1, 0]
        # Reshape to rank-3 tensor with bond dimension 1
        tensor = tf.constant([[1.0], [0.0]], dtype=tf.complex64)  # shape: [2, 1]
        tensor = tf.reshape(tensor, [1, 2, 1])  # shape: [1, 2, 1]
        tensors.append(tensor)
    
    # Create FiniteMPS
    mps = tn.FiniteMPS(tensors, canonicalize=True)
    return mps

def create_simple_mpo(n_qubits, operator='Z'):
    """
    Create a simple MPO representing a product of Pauli operators.
    
    Args:
        n_qubits: Number of qubits
        operator: 'X', 'Y', or 'Z' for Pauli operators
    
    Returns:
        FiniteMPO object
    """
    # Define Pauli matrices
    I = tf.constant([[1.0, 0.0], [0.0, 1.0]], dtype=tf.complex64)
    X = tf.constant([[0.0, 1.0], [1.0, 0.0]], dtype=tf.complex64)
    Y = tf.constant([[0.0, -1j], [1j, 0.0]], dtype=tf.complex64)
    Z = tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64)
    
    # Select the operator
    op_dict = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    op = op_dict.get(operator, Z)
    
    # Create MPO tensors
    mpo_tensors = []
    for _ in range(n_qubits):
        # Reshape operator to rank-4 tensor: [1, 1, 2, 2]
        # Dimensions: [left_bond, right_bond, physical_in, physical_out]
        tensor = tf.reshape(op, [1, 1, 2, 2])
        mpo_tensors.append(tensor)
    
    # Create FiniteMPO
    mpo = tn.FiniteMPO(mpo_tensors)
    return mpo

def apply_mpo_to_mps(mps, mpo):
    """
    Apply an MPO to an MPS
    
    Args:
        mps: FiniteMPS object
        mpo: FiniteMPO object
    
    Returns:
        FiniteMPS resulting from applying the MPO to the MPS
    """
    return mpo.apply(mps)

def compute_expectation_value(mps, mpo):
    """
    Compute the expectation value <ψ|O|ψ>
    
    Args:
        mps: FiniteMPS object
        mpo: FiniteMPO object
    
    Returns:
        Expectation value (real part)
    """
    # Apply MPO to MPS
    mps_prime = apply_mpo_to_mps(mps, mpo)
    
    # Compute inner product
    expectation = mps.inner_product(mps_prime)
    
    # Return real part (should be real for Hermitian observables)
    return tf.math.real(expectation)

def apply_single_site_gate(mps, gate, site):
    """
    Apply a single-site gate to an MPS
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2] tensor representing gate
        site: Site to apply gate to
    
    Returns:
        Updated FiniteMPS
    """
    # Get tensor at the site
    tensor = mps.tensors[site]  # [D1, 2, D2]
    
    # Make a copy of all tensors
    new_tensors = [tf.identity(t) for t in mps.tensors]
    
    # Apply gate to the tensor
    # Contract gate with physical dimension of tensor
    new_tensor = tf.einsum('ab,cbd->cad', gate, tensor)
    
    # Update tensor in the list
    new_tensors[site] = new_tensor
    
    # Create new MPS
    new_mps = tn.FiniteMPS(new_tensors, canonicalize=False)
    
    # Copy center position from original MPS
    new_mps.center_position = mps.center_position
    
    return new_mps

def test_mps_mpo_operations():
    """
    Test basic MPS and MPO operations
    """
    # Number of qubits
    n_qubits = 3
    
    print("=== Basic MPS and MPO Operations ===")
    print(f"Number of qubits: {n_qubits}")
    
    # Create MPS in |000⟩ state
    mps = initialize_simple_mps(n_qubits)
    print("\nMPS tensors shapes:")
    for i, tensor in enumerate(mps.tensors):
        print(f"  Tensor {i}: {tensor.shape}")
    
    # Create MPO for Z operator on each site
    mpo_z = create_simple_mpo(n_qubits, 'Z')
    print("\nMPO tensors shapes:")
    for i, tensor in enumerate(mpo_z.tensors):
        print(f"  Tensor {i}: {tensor.shape}")
    
    # Compute expectation value for Z^⊗n
    expectation_z = compute_expectation_value(mps, mpo_z)
    print(f"\nExpectation value <000|Z^⊗{n_qubits}|000>: {expectation_z.numpy()}")
    
    # Define Hadamard gate
    H = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
    
    # Apply Hadamard to first qubit: |000⟩ -> |+00⟩
    mps_h = apply_single_site_gate(mps, H, 0)
    
    # Compute new expectation values
    expectation_z_h = compute_expectation_value(mps_h, mpo_z)
    print(f"\nAfter applying H to first qubit:")
    print(f"Expectation value <+00|Z^⊗{n_qubits}|+00>: {expectation_z_h.numpy()}")
    
    # Create MPO for X operator on each site
    mpo_x = create_simple_mpo(n_qubits, 'X')
    expectation_x_h = compute_expectation_value(mps_h, mpo_x)
    print(f"Expectation value <+00|X^⊗{n_qubits}|+00>: {expectation_x_h.numpy()}")
    
    return mps, mpo_z

def apply_adjacent_two_qubit_gate_simple(mps, gate, site1, site2):
    """
    Apply a two-qubit gate to adjacent sites in an MPS (simplified version)
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2, 2, 2] tensor for two-qubit gate
        site1, site2: Adjacent sites to apply gate to
    
    Returns:
        Updated FiniteMPS
    """
    # Make sure sites are adjacent
    if abs(site1 - site2) != 1:
        raise ValueError("Sites must be adjacent")
    
    # Make sure site1 < site2
    if site1 > site2:
        site1, site2 = site2, site1
        # Swap gate indices
        gate = tf.transpose(gate, [2, 3, 0, 1])
    
    # Get tensors at the sites
    tensor1 = mps.tensors[site1]  # [D1, 2, D2]
    tensor2 = mps.tensors[site2]  # [D2, 2, D3]
    
    # Extract dimensions
    D1, _, D2 = tensor1.shape
    _, _, D3 = tensor2.shape
    
    # Combine the two tensors into a single tensor
    # First, reshape tensors for appropriate contraction
    tensor1_reshaped = tf.reshape(tensor1, [D1, 2, D2])
    tensor2_reshaped = tf.reshape(tensor2, [D2, 2, D3])
    
    # Contract along the bond dimension
    combined = tf.einsum('abc,cde->abde', tensor1_reshaped, tensor2_reshaped)
    # Result shape: [D1, 2, 2, D3]
    
    # Apply the two-qubit gate
    updated = tf.einsum('abcd,bcej->aejd', combined, gate)
    # Result shape: [D1, 2, 2, D3]
    
    # Make a copy of all tensors
    new_tensors = [tf.identity(t) for t in mps.tensors]
    
    # For simplicity, we'll decompose the tensor using simple reshaping
    # This is a simplified approach and doesn't preserve canonical form or truncate
    
    # Simply reshape the contracted tensor
    s1 = tf.reshape(updated, [D1, 2, 2 * D3])
    new_tensors[site1] = s1
    
    # Create s2 as a dummy tensor to maintain the MPS structure
    # Not a proper SVD decomposition, but works for demonstration
    s2 = tf.ones([2 * D3, 2, D3], dtype=tf.complex64)
    s2 = s2 / tf.cast(tf.sqrt(tf.cast(2 * D3, tf.float32)), tf.complex64)  # Normalize
    new_tensors[site2] = s2
    
    # Create new MPS
    new_mps = tn.FiniteMPS(new_tensors, canonicalize=False)
    
    # Set center position
    new_mps.center_position = mps.center_position
    
    return new_mps

def test_two_qubit_gate():
    """
    Test applying a two-qubit gate to an MPS
    """
    # Number of qubits
    n_qubits = 3
    
    print("\n=== Two-Qubit Gate Application ===")
    
    # Create MPS in |000⟩ state
    mps = initialize_simple_mps(n_qubits)
    
    # Define CNOT gate as a two-qubit gate
    # CNOT = |0⟩⟨0| ⊗ I + |1⟩⟨1| ⊗ X
    CNOT = tf.zeros([2, 2, 2, 2], dtype=tf.complex64)
    
    # |00⟩ -> |00⟩
    CNOT = tf.tensor_scatter_nd_update(CNOT, [[0, 0, 0, 0]], [tf.complex(1.0, 0.0)])
    # |01⟩ -> |01⟩
    CNOT = tf.tensor_scatter_nd_update(CNOT, [[0, 1, 0, 1]], [tf.complex(1.0, 0.0)])
    # |10⟩ -> |11⟩
    CNOT = tf.tensor_scatter_nd_update(CNOT, [[1, 0, 1, 1]], [tf.complex(1.0, 0.0)])
    # |11⟩ -> |10⟩
    CNOT = tf.tensor_scatter_nd_update(CNOT, [[1, 1, 1, 0]], [tf.complex(1.0, 0.0)])
    
    # Apply CNOT to qubits 0 and 1
    mps_cnot = apply_adjacent_two_qubit_gate_simple(mps, CNOT, 0, 1)
    
    # Create Z operator MPO
    mpo_z = create_simple_mpo(n_qubits, 'Z')
    
    # Calculate expectation value
    expectation = compute_expectation_value(mps_cnot, mpo_z)
    print(f"Expectation value after CNOT: {expectation.numpy()}")
    
    # Define Hadamard gate
    H = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
    
    # Apply Hadamard to first qubit
    mps_h = apply_single_site_gate(mps, H, 0)
    
    # Apply CNOT to H|0⟩ ⊗ |0⟩ = |+⟩|0⟩
    mps_bell = apply_adjacent_two_qubit_gate_simple(mps_h, CNOT, 0, 1)
    
    # Measure X on first qubit and Z on second qubit
    mpo_x_1 = create_simple_mpo(n_qubits, 'I')  # Start with identity
    mpo_x_1.tensors[0] = tf.reshape(tf.constant([[0.0, 1.0], [1.0, 0.0]], dtype=tf.complex64), [1, 1, 2, 2])
    
    mpo_z_2 = create_simple_mpo(n_qubits, 'I')  # Start with identity
    mpo_z_2.tensors[1] = tf.reshape(tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64), [1, 1, 2, 2])
    
    # Calculate expectation values
    exp_x1 = compute_expectation_value(mps_bell, mpo_x_1)
    exp_z2 = compute_expectation_value(mps_bell, mpo_z_2)
    
    print(f"Bell state X₁ expectation: {exp_x1.numpy()}")
    print(f"Bell state Z₂ expectation: {exp_z2.numpy()}")
    
    return mps_bell

def main():
    """
    Main function to run tests
    """
    # Test basic MPS and MPO operations
    mps, mpo = test_mps_mpo_operations()
    
    # Test two-qubit gate application
    bell_state = test_two_qubit_gate()
    
    print("\nAll tests completed successfully!")
    return mps, mpo, bell_state

if __name__ == "__main__":
    main()

=== Basic MPS and MPO Operations ===
Number of qubits: 3

MPS tensors shapes:
  Tensor 0: (1, 2, 1)
  Tensor 1: (1, 2, 1)
  Tensor 2: (1, 2, 1)


ValueError: Can't convert Python sequence with mixed types to Tensor.

In [ ]:
import numpy as np
import tensorflow as tf
import tensornetwork as tn

# Set TensorNetwork to use TensorFlow backend
tn.set_default_backend("tensorflow")

def initialize_simple_mps(n_qubits):
    """
    Initialize a simple MPS with all qubits in |0⟩
    """
    # Create tensors for MPS in |0⟩^⊗n state
    tensors = []
    for _ in range(n_qubits):
        # Create tensor for |0⟩ state: [1, 0]
        # Reshape to rank-3 tensor with bond dimension 1
        tensor = tf.constant([[1.0], [0.0]], dtype=tf.complex64)  # shape: [2, 1]
        tensor = tf.reshape(tensor, [1, 2, 1])  # shape: [1, 2, 1]
        tensors.append(tensor)
    
    # Create FiniteMPS
    mps = tn.FiniteMPS(tensors, canonicalize=True)
    return mps

def create_simple_mpo(n_qubits, operator='Z'):
    """
    Create a simple MPO representing a product of Pauli operators.
    
    Args:
        n_qubits: Number of qubits
        operator: 'X', 'Y', or 'Z' for Pauli operators
    
    Returns:
        FiniteMPO object
    """
    # Define Pauli matrices
    I = tf.constant([[1.0, 0.0], [0.0, 1.0]], dtype=tf.complex64)
    X = tf.constant([[0.0, 1.0], [1.0, 0.0]], dtype=tf.complex64)
    Y = tf.constant([[0.0, -1j], [1j, 0.0]], dtype=tf.complex64)
    Z = tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64)
    
    # Select the operator
    op_dict = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    op = op_dict.get(operator, Z)
    
    # Create MPO tensors
    mpo_tensors = []
    for _ in range(n_qubits):
        # Reshape operator to rank-4 tensor: [1, 1, 2, 2]
        # Dimensions: [left_bond, right_bond, physical_in, physical_out]
        tensor = tf.reshape(op, [1, 1, 2, 2])
        mpo_tensors.append(tensor)
    
    # Create FiniteMPO
    mpo = tn.FiniteMPO(mpo_tensors)
    return mpo

def apply_mpo_to_mps(mps, mpo):
    """
    Apply an MPO to an MPS
    
    Args:
        mps: FiniteMPS object
        mpo: FiniteMPO object
    
    Returns:
        FiniteMPS resulting from applying the MPO to the MPS
    """
    return mpo.apply(mps)

def compute_expectation_value(mps, mpo):
    """
    Compute the expectation value <ψ|O|ψ>
    
    Args:
        mps: FiniteMPS object
        mpo: FiniteMPO object
    
    Returns:
        Expectation value (real part)
    """
    # Apply MPO to MPS
    mps_prime = apply_mpo_to_mps(mps, mpo)
    
    # Compute inner product
    expectation = mps.inner_product(mps_prime)
    
    # Return real part (should be real for Hermitian observables)
    return tf.math.real(expectation)

def apply_single_site_gate(mps, gate, site):
    """
    Apply a single-site gate to an MPS
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2] tensor representing gate
        site: Site to apply gate to
    
    Returns:
        Updated FiniteMPS
    """
    # Get tensor at the site
    tensor = mps.tensors[site]  # [D1, 2, D2]
    
    # Make a copy of all tensors
    new_tensors = [tf.identity(t) for t in mps.tensors]
    
    # Apply gate to the tensor
    # Contract gate with physical dimension of tensor
    new_tensor = tf.einsum('ab,cbd->cad', gate, tensor)
    
    # Update tensor in the list
    new_tensors[site] = new_tensor
    
    # Create new MPS
    new_mps = tn.FiniteMPS(new_tensors, canonicalize=False)
    
    # Copy center position from original MPS
    new_mps.center_position = mps.center_position
    
    return new_mps

def test_mps_mpo_operations():
    """
    Test basic MPS and MPO operations
    """
    # Number of qubits
    n_qubits = 3
    
    print("=== Basic MPS and MPO Operations ===")
    print(f"Number of qubits: {n_qubits}")
    
    # Create MPS in |000⟩ state
    mps = initialize_simple_mps(n_qubits)
    print("\nMPS tensors shapes:")
    for i, tensor in enumerate(mps.tensors):
        print(f"  Tensor {i}: {tensor.shape}")
    
    # Create MPO for Z operator on each site
    mpo_z = create_simple_mpo(n_qubits, 'Z')
    print("\nMPO tensors shapes:")
    for i, tensor in enumerate(mpo_z.tensors):
        print(f"  Tensor {i}: {tensor.shape}")
    
    # Compute expectation value for Z^⊗n
    expectation_z = compute_expectation_value(mps, mpo_z)
    print(f"\nExpectation value <000|Z^⊗{n_qubits}|000>: {expectation_z.numpy()}")
    
    # Define Hadamard gate
    H = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
    
    # Apply Hadamard to first qubit: |000⟩ -> |+00⟩
    mps_h = apply_single_site_gate(mps, H, 0)
    
    # Compute new expectation values
    expectation_z_h = compute_expectation_value(mps_h, mpo_z)
    print(f"\nAfter applying H to first qubit:")
    print(f"Expectation value <+00|Z^⊗{n_qubits}|+00>: {expectation_z_h.numpy()}")
    
    # Create MPO for X operator on each site
    mpo_x = create_simple_mpo(n_qubits, 'X')
    expectation_x_h = compute_expectation_value(mps_h, mpo_x)
    print(f"Expectation value <+00|X^⊗{n_qubits}|+00>: {expectation_x_h.numpy()}")
    
    return mps, mpo_z

def apply_adjacent_two_qubit_gate(mps, gate, site1, site2):
    """
    Apply a two-qubit gate to adjacent sites in an MPS using SVD decomposition
    
    Args:
        mps: FiniteMPS object
        gate: [2, 2, 2, 2] tensor for two-qubit gate
        site1, site2: Adjacent sites to apply gate to
    
    Returns:
        Updated FiniteMPS
    """
    # Make sure sites are adjacent
    if abs(site1 - site2) != 1:
        raise ValueError("Sites must be adjacent")
    
    # Make sure site1 < site2
    if site1 > site2:
        site1, site2 = site2, site1
        # Swap gate indices
        gate = tf.transpose(gate, [2, 3, 0, 1])
    
    # Get tensors at the sites
    tensor1 = mps.tensors[site1]  # [D1, 2, D2]
    tensor2 = mps.tensors[site2]  # [D2, 2, D3]
    
    # Extract dimensions
    D1, d1, D2 = tensor1.shape
    _, d2, D3 = tensor2.shape
    
    # Combine the two tensors into a single tensor using einsum
    combined = tf.einsum('abc,cde->abde', tensor1, tensor2)
    # Result shape: [D1, 2, 2, D3]
    
    # Apply the two-qubit gate
    combined_after_gate = tf.einsum('abcd,bcej->aejd', combined, gate)
    # Result shape: [D1, 2, 2, D3]
    
    # Reshape the tensor for SVD
    combined_reshaped = tf.reshape(combined_after_gate, [D1 * 2, 2 * D3])
    
    # Perform SVD
    s, u, v = tf.linalg.svd(combined_reshaped, full_matrices=False)
    
    # Decide how many singular values to keep
    # For this simple example, we'll keep all of them
    chi = s.shape[0]
    
    # Apply the singular values to u and v
    sqrt_s = tf.sqrt(s)
    u_s = u * tf.reshape(sqrt_s, [1, -1])
    v_s = v * tf.reshape(sqrt_s, [-1, 1])
    
    # Reshape back to MPS tensor format
    new_tensor1 = tf.reshape(u_s, [D1, 2, chi])
    new_tensor2 = tf.reshape(v_s, [chi, 2, D3])
    
    # Create new tensors list
    new_tensors = [tf.identity(t) for t in mps.tensors]
    new_tensors[site1] = new_tensor1
    new_tensors[site2] = new_tensor2
    
    # Create new MPS
    new_mps = tn.FiniteMPS(new_tensors, canonicalize=False)
    
    # Set center position
    new_mps.center_position = site1
    
    return new_mps

def prepare_cnot_gate():
    """
    Prepare a CNOT gate tensor
    
    Returns:
        CNOT gate as a [2, 2, 2, 2] tensor
    """
    # Initialize CNOT tensor with zeros
    cnot = tf.zeros([2, 2, 2, 2], dtype=tf.complex64)
    
    # Define the CNOT gate in computational basis
    # |00⟩ -> |00⟩
    cnot = tf.tensor_scatter_nd_update(
        cnot, [[0, 0, 0, 0]], 
        [tf.constant(1.0, dtype=tf.complex64)]
    )
    
    # |01⟩ -> |01⟩
    cnot = tf.tensor_scatter_nd_update(
        cnot, [[0, 1, 0, 1]], 
        [tf.constant(1.0, dtype=tf.complex64)]
    )
    
    # |10⟩ -> |11⟩
    cnot = tf.tensor_scatter_nd_update(
        cnot, [[1, 0, 1, 1]], 
        [tf.constant(1.0, dtype=tf.complex64)]
    )
    
    # |11⟩ -> |10⟩
    cnot = tf.tensor_scatter_nd_update(
        cnot, [[1, 1, 1, 0]], 
        [tf.constant(1.0, dtype=tf.complex64)]
    )
    
    return cnot

def test_two_qubit_gate():
    """
    Test applying a two-qubit gate to an MPS
    """
    # Number of qubits
    n_qubits = 3
    
    print("\n=== Two-Qubit Gate Application ===")
    
    # Create MPS in |000⟩ state
    mps = initialize_simple_mps(n_qubits)
    
    # Prepare CNOT gate
    cnot = prepare_cnot_gate()
    
    # Apply CNOT to qubits 0 and 1
    mps_cnot = apply_adjacent_two_qubit_gate(mps, cnot, 0, 1)
    
    # Create Z operator MPO
    mpo_z1 = create_simple_mpo(n_qubits, 'I')  # Start with identity
    mpo_z1.tensors[0] = tf.reshape(
        tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64), 
        [1, 1, 2, 2]
    )
    
    # Calculate expectation value
    expectation = compute_expectation_value(mps_cnot, mpo_z1)
    print(f"Expectation value <ψ|Z₁|ψ> after CNOT on |000⟩: {expectation.numpy()}")
    
    # Define Hadamard gate
    H = tf.constant([[1.0, 1.0], [1.0, -1.0]], dtype=tf.complex64) / tf.sqrt(tf.cast(2.0, tf.complex64))
    
    # Apply Hadamard to first qubit
    mps_h = apply_single_site_gate(mps, H, 0)
    
    # Apply CNOT to H|0⟩ ⊗ |0⟩ = |+⟩|0⟩ to create Bell state
    mps_bell = apply_adjacent_two_qubit_gate(mps_h, cnot, 0, 1)
    
    # Measure X on first qubit
    mpo_x1 = create_simple_mpo(n_qubits, 'I')  # Start with identity
    mpo_x1.tensors[0] = tf.reshape(
        tf.constant([[0.0, 1.0], [1.0, 0.0]], dtype=tf.complex64), 
        [1, 1, 2, 2]
    )
    
    # Measure Z on second qubit
    mpo_z2 = create_simple_mpo(n_qubits, 'I')  # Start with identity
    mpo_z2.tensors[1] = tf.reshape(
        tf.constant([[1.0, 0.0], [0.0, -1.0]], dtype=tf.complex64), 
        [1, 1, 2, 2]
    )
    
    # Calculate expectation values for Bell state
    exp_x1 = compute_expectation_value(mps_bell, mpo_x1)
    exp_z2 = compute_expectation_value(mps_bell, mpo_z2)
    exp_x1z2 = compute_expectation_value(mps_bell, mpo_x1)  # X₁Z₂ correlation
    
    print(f"Bell state X₁ expectation: {exp_x1.numpy()}")
    print(f"Bell state Z₂ expectation: {exp_z2.numpy()}")
    
    return mps_bell

def main():
    """
    Main function to run tests
    """
    # Test basic MPS and MPO operations
    mps, mpo = test_mps_mpo_operations()
    
    # Test two-qubit gate application
    bell_state = test_two_qubit_gate()
    
    print("\nAll tests completed successfully!")
    return mps, mpo, bell_state

if __name__ == "__main__":
    main()

=== Basic MPS and MPO Operations ===
Number of qubits: 3

MPS tensors shapes:
  Tensor 0: (1, 2, 1)
  Tensor 1: (1, 2, 1)
  Tensor 2: (1, 2, 1)


ValueError: Can't convert Python sequence with mixed types to Tensor.